# 0.0 Imports

In [8]:
import pandas as pd
import numpy  as np


from scipy                       import stats
from scipy.stats                 import shapiro, ttest_1samp, mannwhitneyu, f_oneway
from statsmodels.stats           import api as sms
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 1.0 Parâmetros do Experimento

In [4]:
# Funciona para distribuições normais

# Nível de confiança
confidence_level = 0.95

# Nível de significância
significance_level = 1 - confidence_level

# Conversões da pág atual e da nova pág
p1 = 0.13 # atual
p2 = 0.15 # esperado

# Tamanho do efeito
effect_size = sms.proportion_effectsize(p1, p2) 

# Poder estatístico
power = 0.80

In [5]:
# Sample size
sample_n = sms.NormalIndPower().solve_power(
                effect_size,
                power=power,
                alpha=significance_level
)

In [6]:
sample_n = np.round(sample_n, 0).astype(int)
sample_n

4719

In [7]:
print(f'O tamamanho total da amostra é {2*sample_n}')
print(f'O tamamanho da amostra do grupo controle é de {sample_n}')
print(f'O tamamanho da amostra do grupo de tratamento é de {sample_n}')

O tamamanho total da amostra é 9438
O tamamanho da amostra do grupo controle é de 4719
O tamamanho da amostra do grupo de tratamento é de 4719


# 2.0 Testes Estatísticos

## 2.1 One sample test

In [9]:
# One Sample t-test

grupo = [14, 14, 16, 13, 12, 17, 15, 14, 15, 13, 15, 14]

# H0: media = 15
# H1: media != 15

statstic, pvalue = ttest_1samp(grupo, popmean=15)
print(f'p-value: {pvalue}')

p-value: 0.12014460742498101


In [10]:
# p-value < alpha: Rejeita H0, aceita H1
# p-value > alpha: Falha em rejeitar H0 
    # não existe evidência suficiente para rejeitar H0

## 2.2 Two sample t-test (Condições paramétricas satisfeitas)

In [12]:
df1 = [14, 14, 16, 13, 12, 17, 15, 14, 15, 13, 15, 14, 14, 14, 16, 13, 12, 17, 15, 14, 15, 13, 15, 14]
df2 = [12, 17, 15, 14, 12, 17, 15, 14, 13, 13, 16, 15, 15, 13, 15, 14, 14, 13, 13, 16, 15, 15, 12, 15]

In [13]:
np.mean(df1).round(2)

14.33

In [14]:
np.mean(df2).round(2)

14.29

In [15]:
# Shapiro
# H0: distribuição normal

# p-value < alpha: Rejeita H0, aceita H1
# p-value > alpha: Falha em rejeitar H0 

In [16]:
stats1, pvalue1 = shapiro(df1)
print(f'p-value1: {pvalue1}')

stats2, pvalue2 = shapiro(df2)
print(f'p-value2: {pvalue2}')

p-value1: 0.15192988514900208
p-value2: 0.1016923263669014


In [17]:
# H0: u1 = u2        # a altura das plantas é igual
# H1: u1 <> u2       # não tenho evidência suficiente pra falar que o adubo foi o responsável pelas médias estarem diferentes

# p-value < alpha: Rejeita H0

In [18]:
stats, pvalue = stats.ttest_ind(df1, df2, equal_var=True)
pvalue

0.9183924978713519

## 2.3 Mann-Whitney U-test (Condições paramétricas não satisfeitas)

In [19]:
df1 = [20, 23, 21, 20, 18, 17, 18, 14, 60, 24, 48, 19]
df2 = [24, 25, 21, 22, 23, 18, 17, 28, 24, 27, 21, 23]

In [20]:
np.mean(df1)

25.166666666666668

In [21]:
np.mean(df2)

22.75

In [22]:
# H0: u1 = u2        # a altura das plantas é igual
# H1: u1 <> u2       # não tenho evidência suficiente pra falar que o adubo foi o responsável pelas médias estarem diferentes

In [23]:
# Condições paramétricas:

# 1. Amostras independentes
# 2. Normalmente distribuídos
    # Shapiro
        # H0: normalmente distribuídos
        # H1: não normal
        
        # if p-value < alpha:
            # Rejeito H0
            # pvalue = 0.0001 então não é normal
        # else:
            # Falha em rejeitar H0
            # pvalue = 0.25 é normal

In [24]:
stats1, pvalue1 = shapiro(df1)
print(f'p-value1: {pvalue1}')

stats2, pvalue2 = shapiro(df2)
print(f'p-value2: {pvalue2}')
        
# 3. Variâncias iguais
    # v1 / v2 < 4 então OK
    
var1 = np.var(df1).round(2)
var2 = np.var(df2).round(2)
print(f'var1: {var1}\nvar2: {var2}\nvar1/var2: {(var1/var2).round(2)}')

p-value1: 0.00043336229282431304
p-value2: 0.9021502137184143
var1: 178.64
var2: 9.69
var1/var2: 18.44


In [25]:
stats, pvalue = mannwhitneyu(df1, df2)
print(f'p-value: {pvalue}')

p-value: 0.22352512203904684


In [26]:
# H0: u1 = u2        # a altura das plantas é igual
# H1: u1 <> u2       # não tenho evidência suficiente pra falar que o adubo foi o responsável pelas médias estarem diferentes

# p-value < alpha:
    # Rejeito H0

## 2.4 ANOVA (mais que 2 grupos)

In [27]:
df = pd.DataFrame({'A': [25, 30, 28, 36, 29],
                  'B': [45, 55, 29, 56, 40],
                  'C': [30, 29, 33, 37, 27],
                  'D': [54, 60, 51, 62, 73]})

In [28]:
df_melt = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['A','B','C','D'])
df_melt.columns = ['index','treatment','value']
df_melt.head(10)

,index,treatment,value
0,0,A,25
1,1,A,30
2,2,A,28
3,3,A,36
4,4,A,29
5,0,B,45
6,1,B,55
7,2,B,29
8,3,B,56
9,4,B,40


In [31]:
statsA, pvalueA = shapiro(df_melt['value'][df_melt['treatment'] == 'A'])
pvalueA = np.round(pvalueA, 2)

statsB, pvalueB = shapiro(df_melt['value'][df_melt['treatment'] == 'B'])
pvalueB = np.round(pvalueB, 2)

statsC, pvalueC = shapiro(df_melt['value'][df_melt['treatment'] == 'C'])
pvalueC = np.round(pvalueC, 2)

statsD, pvalueD = shapiro(df_melt['value'][df_melt['treatment'] == 'D'])
pvalueD = np.round(pvalueD, 2)

In [32]:
# Shapiro
    # H0: distribuídos normalmente
    # if p-value < alpha:
        # Rejeita H0

print(f'pvalue - Grupo A: {pvalueA}')
print(f'pvalue - Grupo B: {pvalueB}')
print(f'pvalue - Grupo C: {pvalueC}')
print(f'pvalue - Grupo D: {pvalueD}')

# Como p-value > 0.05 então as distribuições são normais

pvalue - Grupo A: 0.61
pvalue - Grupo B: 0.57
pvalue - Grupo C: 0.76
pvalue - Grupo D: 0.7


In [33]:
a = df['A']
b = df['B']
c = df['C']
d = df['D']

In [34]:
print(f'var - Grupo A: {np.var(a).round(2)}')
print(f'var - Grupo B: {np.var(b)}')
print(f'var - Grupo C: {np.var(c)}')
print(f'var - Grupo D: {np.var(d)}')

var - Grupo A: 13.04
var - Grupo B: 100.4
var - Grupo C: 12.16
var - Grupo D: 58.0


In [35]:
stats, pvalue = f_oneway(df['A'], df['B'], df['C'], df['D'])
pvalue

2.639241146210922e-05

In [36]:
# H0: não há diferença entre os grupo

# if pvalue < alpha: rejeita H0               
    # ou seja, há diferença entre os grupos

## 2.5 Pairwise Tukey (mais que 2 grupos)

In [37]:
tukey = pairwise_tukeyhsd(endog=df_melt['value'],
                         groups=df_melt['treatment'],
                         alpha=0.05)
print(tukey)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     A      B     15.4 0.0251   1.694 29.106   True
     A      C      1.6 0.9867 -12.106 15.306  False
     A      D     30.4 0.0001  16.694 44.106   True
     B      C    -13.8 0.0482 -27.506 -0.094   True
     B      D     15.0 0.0296   1.294 28.706   True
     C      D     28.8 0.0001  15.094 42.506   True
---------------------------------------------------


## 2.6 Chi-square

In [81]:
from scipy.stats import chi2_contingency
#chi_val, pval, dof, expected = chi2_contingency(df)